In [1]:
import os, sys
parent_dir = os.path.abspath('../..')

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import

In [2]:
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.utils import make_grid
import torchvision.datasets as dset
import matplotlib.pyplot as plt

import utils_os
import mi
from datasets.mnist_m import MNISTM


torch.manual_seed(time.time())

%load_ext autoreload
%autoreload 2

# Global Variables

In [3]:
DEVICE = 'cuda:0' 

class Hyperparams(utils_os.ConfigDict):

    def __init__(self): 
        self.device = DEVICE
        self.learning_rate = 1e-3
        self.n_epochs = 80
        self.grad_clip = None
        self.batch_size = 256
        self.image_shape = (32, 32)
        self.update_inner_every_num_epoch = 2

BASE_HYPERPARAMS = Hyperparams()

ROOT = '../..'
MODEL_SAVE_DIR = f'{ROOT}/results/m_mm'
DATASET_DIR = f'{ROOT}/data'

NUM_COLOR_CHANNELS = 3
NUM_CLASSES = 10
NUM_DOMAIN = 2

DIM_Z_CONTENT, DIM_Z_DOMAIN = 128, 128

# Prepare Dataloaders

### Load MNIST

In [4]:
if not os.path.exists(DATASET_DIR):
    os.mkdir(DATASET_DIR)
    
trans = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), lambda x: x.repeat(3, 1, 1)])  # expand color channel of grayscale image to align with mnistm
m_train_set = dset.MNIST(root=DATASET_DIR, train=True, transform=trans, download=True)
m_test_set = dset.MNIST(root=DATASET_DIR, train=False, transform=trans, download=True)

m_train_loader = torch.utils.data.DataLoader(
                 dataset=m_train_set,
                 batch_size=BASE_HYPERPARAMS.batch_size)
m_test_loader = torch.utils.data.DataLoader(
                dataset=m_test_set,
                batch_size=BASE_HYPERPARAMS.batch_size,
                shuffle=False)              

DOMAIN1 = 'MNIST'

### Load MNISTM

In [5]:
trans = transforms.Compose([transforms.Resize(32), transforms.ToTensor()])
mm_train_set = MNISTM(root=DATASET_DIR, train=True, transform=trans, download=True)
mm_test_set = MNISTM(root=DATASET_DIR, train=False, transform=trans, download=True)

mm_train_loader = torch.utils.data.DataLoader(
                 dataset=mm_train_set,
                 batch_size=BASE_HYPERPARAMS.batch_size)
mm_test_loader = torch.utils.data.DataLoader(
                dataset=mm_test_set,
                batch_size=BASE_HYPERPARAMS.batch_size,
                shuffle=False)  

DOMAIN2 = 'MNISTM'

../../data/MNISTM/processed/mnist_m_train.pt
../../data/MNISTM/processed/mnist_m_test.pt


### Load all data into memeory for faster access when training infomin layers

In [6]:
def loader2tensor(dataloader):
    all_data, all_target = [], []
    for data, target in dataloader:
        all_data.append(data)
        all_target.append(target)
    print('loader2tensor finished.', 'data max=', data.max(), 'data min=', data.min())
    return torch.cat(all_data, dim=0), torch.cat(all_target, dim=0)

all_data_m, all_label_m = loader2tensor(m_train_loader)
all_data_test_m, all_label_test_m = loader2tensor(m_test_loader)

all_data_mm, all_label_mm = loader2tensor(mm_train_loader)
all_data_test_mm, all_label_test_mm = loader2tensor(mm_test_loader)

INFOMIN_TRAIN_BATCH_SIZE = 2500

loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)


# Experiments

In [7]:
from tasks.domain_adaptation.experiment import DomainAdaptation, train, test, exp_run


def inner_batch_provider():
    idx1, idx2 = torch.randperm(len(all_data_m)), torch.randperm(len(all_data_mm))
    infomin_x1, infomin_x2 = all_data_m[idx1[:INFOMIN_TRAIN_BATCH_SIZE]].to(DEVICE), all_data_mm[idx2[:INFOMIN_TRAIN_BATCH_SIZE]].to(DEVICE)

    return infomin_x1, infomin_x2


def get_model_path(hyperparams, epoch=None):
    if epoch is not None:
        return f'{MODEL_SAVE_DIR}/{hyperparams.estimator}/model_{hyperparams.get_name("alpha", "beta", "gamma")}_{epoch}'
    return f'{MODEL_SAVE_DIR}/{hyperparams.estimator}/model_{hyperparams.get_name("alpha", "beta", "gamma")}'

In [8]:
import copy

hyperparams = copy.deepcopy(BASE_HYPERPARAMS)
hyperparams.alpha = 1.0
hyperparams.beta = 1.0
hyperparams.gamma = 0.1
hyperparams.n_slice = 100
hyperparams.estimator = 'SLICE'

da = DomainAdaptation(DIM_Z_CONTENT, DIM_Z_DOMAIN, NUM_CLASSES, NUM_DOMAIN, hyperparams).to(DEVICE)

da = exp_run(
    (m_train_loader, mm_train_loader), (m_test_loader, mm_test_loader),
    train, test,
    inner_batch_provider, get_model_path,
    hyperparams,
    device=DEVICE,
    model=da,
    scheduler_func=lambda optimizer: torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
)

epoch: 0, loss: 0.04842029077699408, loss_content: 0.04325547500047833, loss_domain: 0.0012259285402251407 redundancy: 0.03938886962132528  acc_d1: 98.75999450683594 acc_d2: 76.48999786376953 acc_domain: 99.97999572753906

epoch: 1, loss: 0.04836786184459925, loss_content: 0.043369650936801916, loss_domain: 0.0003613408229284687 redundancy: 0.04636870144167915  acc_d1: 98.75 acc_d2: 78.95999908447266 acc_domain: 100.0

epoch: 2, loss: 0.052135483757592735, loss_content: 0.03297151937731542, loss_domain: 0.0006638608188950457 redundancy: 0.18500103503465654  acc_d1: 99.0 acc_d2: 80.6500015258789 acc_domain: 99.9749984741211

epoch: 3, loss: 0.039750883809756485, loss_content: 0.032433102950744794, loss_domain: 8.646684873383492e-05 redundancy: 0.0723131388425827  acc_d1: 99.00999450683594 acc_d2: 82.08999633789062 acc_domain: 100.0

epoch: 4, loss: 0.03729435769491829, loss_content: 0.026307215871929655, loss_domain: 0.0026576524833217263 redundancy: 0.08329489259049297  acc_d1: 99.3499

In [9]:
import copy

hyperparams = copy.deepcopy(BASE_HYPERPARAMS)
hyperparams.alpha = 1.0
hyperparams.beta = 1.0
hyperparams.gamma = 0.01
hyperparams.estimator = 'CLUB'
hyperparams.learning_rate = 1e-4
hyperparams.inner_lr = 5e-4
hyperparams.inner_epochs = 5
hyperparams.inner_batch_size = 2500

da = DomainAdaptation(DIM_Z_CONTENT, DIM_Z_DOMAIN, NUM_CLASSES, NUM_DOMAIN, hyperparams).to(DEVICE)

da = exp_run(
    (m_train_loader, mm_train_loader), (m_test_loader, mm_test_loader),
    train, test,
    inner_batch_provider, get_model_path,
    hyperparams,
    device=DEVICE,
    model=da,
    scheduler_func=lambda optimizer: torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
)

epoch: 0, loss: 0.15741917621344328, loss_content: 0.14290736336261034, loss_domain: 0.0040685663116164505 redundancy: 1.044324666261673  acc_d1: 98.44999694824219 acc_d2: 46.28999710083008 acc_domain: 99.9949951171875

epoch: 1, loss: 0.1541142975911498, loss_content: 0.1513977523893118, loss_domain: 0.0016758469166234136 redundancy: 0.10406996756792068  acc_d1: 98.90999603271484 acc_d2: 58.119998931884766 acc_domain: 99.9949951171875

epoch: 2, loss: 0.09003711966797709, loss_content: 0.08174250470474362, loss_domain: 0.000926472683204338 redundancy: 0.7368142738938331  acc_d1: 99.29000091552734 acc_d2: 60.63999938964844 acc_domain: 99.9949951171875

epoch: 3, loss: 0.12130973841995, loss_content: 0.11699931845068931, loss_domain: 0.0005992830410832539 redundancy: 0.37111364752054216  acc_d1: 97.48999786376953 acc_d2: 60.71999740600586 acc_domain: 99.9949951171875

epoch: 4, loss: 0.07886816142126918, loss_content: 0.07654108223505318, loss_domain: 0.0004207506986858789 redundancy: 0